In [1]:
import numpy as np
import pandas as pd

In [2]:
movie=pd.read_csv('tmdb_5000_movies.csv')
credit=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
# merging on the basis of title
movie=movie.merge(credit,on='title')

In [6]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
# content base merging columns
# genre, id, keywords, overview, cast, crew, title
movie['original_language'].value_counts()
# as 95% of lang is en ; so it'll make the data unbalanced. Therefore ignore it.

original_language
en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
id       2
cs       2
ta       2
ro       2
ar       2
te       1
hu       1
xx       1
af       1
is       1
tr       1
vi       1
pl       1
nb       1
ky       1
no       1
sl       1
ps       1
el       1
Name: count, dtype: int64

In [8]:
movie=movie[['id','title','overview','keywords','genres','cast','crew']]

In [9]:
movie.head(1)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
movie.isnull().sum()

id          0
title       0
overview    3
keywords    0
genres      0
cast        0
crew        0
dtype: int64

In [11]:
movie.dropna(inplace=True)

In [12]:
# convert (overview keywords genres	cast crew) to a paragh. First convert the list of dict to list of string.
import ast
# has a fun which converts string to list

In [13]:
def convert(text):
    L=[]
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L    

In [14]:
movie['keywords']=movie['keywords'].apply(convert)

In [15]:
movie['genres']=movie['genres'].apply(convert)

In [16]:
movie.head(1)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [17]:
# we want only top 3 members of cast

def cast(text):
    L=[]
    c=0
    for i in ast.literal_eval(text):
        if c!=3:
            L.append(i['name'])
            c+=1
        else : break    
    return L

In [18]:
movie['cast']=movie['cast'].apply(cast)

In [19]:
# for crew we want only director
def director(text):
    L=[]
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            L.append(i['name'])
    return L        

In [20]:
movie['crew']=movie['crew'].apply(director)

In [21]:
movie.head(1)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [22]:
# now convert Sam Worthington -> SamWorthington (remove space); for all col
movie['keywords']= movie['keywords'].apply(lambda x :[i.replace(" ","") for i in x])
movie['genres']= movie['genres'].apply(lambda x :[i.replace(" ","") for i in x])
movie['cast']= movie['cast'].apply(lambda x :[i.replace(" ","") for i in x])
movie['crew']= movie['crew'].apply(lambda x :[i.replace(" ","") for i in x])

In [23]:
movie.head(1)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [24]:
def safe_split(item):
    if isinstance(item, str):
        return item.split()
    else:
        return str(item).split()

In [25]:
movie['overview'] = movie['overview'].apply(safe_split)

In [26]:
movie['tag']=movie['overview']+movie['keywords']+movie['genres']+movie['cast']+movie['crew']

In [27]:
movie.head(5)

,id,title,overview,keywords,genres,cast,crew,tag
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [28]:
new_df= movie[['id','title','tag']]

In [29]:
new_df.head(1)

,id,title,tag
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [30]:
# convert tag to single line
new_df['tag']=new_df['tag'].apply(lambda x:" ".join(x))

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13732\3633989473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag']=new_df['tag'].apply(lambda x:" ".join(x))


In [31]:
new_df.head(1)

,id,title,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [32]:
new_df['tag'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d Action Adventure Fantasy ScienceFiction SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [33]:
# recommended to convert all tags to lower case
new_df['tag']=new_df['tag'].apply(lambda x:x.lower())

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13732\1644574083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag']=new_df['tag'].apply(lambda x:x.lower())


In [34]:
new_df['tag'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver jamescameron'

Vectorization

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
# stop_words removes is,at,the,if from tags


In [36]:
vectors = cv.fit_transform(new_df['tag']).toarray()

In [37]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(5000,))

it is array of words used (5000 words). Like if a word is used it is marked 1 ; else 0

In [38]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      shape=(5000,), dtype=object)

In [41]:
import nltk

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [42]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [43]:
new_df['tag']=new_df['tag'].apply(stem)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13732\599176683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag']=new_df['tag'].apply(stem)


In [44]:
# we will take cosine dist b/w the two vectors and the most closest will be the recommendation 
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
#cosine_similarity(vectors) will give dist of every vector from  other vector 
same=cosine_similarity(vectors)

In [46]:
# hence the diagonal element of this matrix will be 1 
same[0]

array([1.        , 0.08858079, 0.05905386, ..., 0.02478408, 0.02599376,
       0.        ], shape=(4806,))

In [57]:
# sort the list(make it a list); but it will give many 0 on top. So we will reverse it. Then enumarate index with the movies so that after sorting also we can fetch
# them. 'key=lambda x:x[1]' applies the whole fun to the second col(as first col is the idx; and we want sorting based on the similarity) 
sorted(list(enumerate(same[0])),reverse=True,key=lambda x:x[1])[1:6]
# the first one is the movie itself ; so donot count it. Instead count the movies from rank 2 to 6

[(539, np.float64(0.2537477434955704)),
 (1192, np.float64(0.25112360116696136)),
 (260, np.float64(0.2442250043490654)),
 (1214, np.float64(0.24260844762468367)),
 (507, np.float64(0.24165612634006073))]

In [67]:
# this fun gives top 5 movies( 5 nearest vectors)
def recommend(movie):
    movie_index= new_df[new_df['title']==movie].index[0]
    dist=same[movie_index]
    movies_list=sorted(list(enumerate(dist)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [69]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman
Batman & Robin


In [62]:
# convert the index to movie title
new_df.iloc[539].title

'Titan A.E.'

In [73]:
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))

In [78]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [80]:
pickle.dump(same,open('same.pkl','wb'))